In [15]:
import requests
import datetime

import json
import pandas as pd
pd.set_option('display.max_rows', 2000)

## Get all district IDs

In [25]:
district_df_all = None
for state_code in range(1,40):
    response = requests.get("https://cdn-api.co-vin.in/api/v2/admin/location/districts/{}".format(state_code))
    district_df = pd.DataFrame(json.loads(response.text))
    district_df = pd.json_normalize(district_df['districts'])
    if district_df_all is None:
        district_df_all = district_df
    else:
        district_df_all = pd.concat([district_df_all, district_df])
        
    district_df_all.district_id = district_df_all.district_id.astype(int)
        
district_df_all = district_df_all[["district_name", "district_id"]].sort_values("district_name")
district_df_all.to_csv("districts.csv", index=False)

## Fix your district id 

In [17]:
Ahmedabad = 154
Ahmedabad_Corporation = 770
DIST_IDS = [Ahmedabad, Ahmedabad_Corporation]
next_n_days = 5

In [22]:
base = datetime.datetime.today()
date_list = [base + datetime.timedelta(days=x) for x in range(next_n_days)]
date_str = [x.strftime("%d-%m-%Y") for x in date_list]

all_date_df = None
for INP_DATE in date_str:
    for DIST_ID in DIST_IDS:
        print(f"checking for INP_DATE:{INP_DATE} & DIST_ID:{DIST_ID}")
        URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id={}&date={}".format(DIST_ID, INP_DATE)
        response = requests.get(URL)
        data = json.loads(response.text)['centers']
        df = pd.DataFrame(data)
        df['min_age_limit'] = df.sessions.apply(lambda x: [{d["date"]:d['min_age_limit']} for d in x])
        df = df.explode("min_age_limit")
        df["date"] = df['min_age_limit'].apply(lambda x: list(x.keys())[0])
        df["min_age_limit"] = df['min_age_limit'].apply(lambda x: list(x.values())[0])
        df = df[["date", "min_age_limit", "name", "state_name", "district_name", "block_name", "pincode", "fee_type"]]
        if all_date_df is not None:
            all_date_df = pd.concat([all_date_df, df])
        else:
            all_date_df = df
            
df.drop(["block_name"], axis=1)

checking for INP_DATE:01-05-2021 & DIST_ID:154
checking for INP_DATE:01-05-2021 & DIST_ID:770
checking for INP_DATE:02-05-2021 & DIST_ID:154
checking for INP_DATE:02-05-2021 & DIST_ID:770
checking for INP_DATE:03-05-2021 & DIST_ID:154
checking for INP_DATE:03-05-2021 & DIST_ID:770
checking for INP_DATE:04-05-2021 & DIST_ID:154
checking for INP_DATE:04-05-2021 & DIST_ID:770
checking for INP_DATE:05-05-2021 & DIST_ID:154
checking for INP_DATE:05-05-2021 & DIST_ID:770


,date,min_age_limit,name,state_name,district_name,pincode,fee_type
0,05-05-2021,45,Raikhad UHC,Gujarat,Ahmedabad Corporation,380001,Free
1,05-05-2021,45,LADY CARE WOMENS HOSPITAL,Gujarat,Ahmedabad Corporation,380013,Paid
1,06-05-2021,45,LADY CARE WOMENS HOSPITAL,Gujarat,Ahmedabad Corporation,380013,Paid
1,07-05-2021,45,LADY CARE WOMENS HOSPITAL,Gujarat,Ahmedabad Corporation,380013,Paid
1,08-05-2021,45,LADY CARE WOMENS HOSPITAL,Gujarat,Ahmedabad Corporation,380013,Paid
1,10-05-2021,45,LADY CARE WOMENS HOSPITAL,Gujarat,Ahmedabad Corporation,380013,Paid
1,11-05-2021,45,LADY CARE WOMENS HOSPITAL,Gujarat,Ahmedabad Corporation,380013,Paid
2,05-05-2021,45,AIMS PALDI,Gujarat,Ahmedabad Corporation,380007,Paid
2,06-05-2021,45,AIMS PALDI,Gujarat,Ahmedabad Corporation,380007,Paid
2,07-05-2021,45,AIMS PALDI,Gujarat,Ahmedabad Corporation,380007,Paid


In [20]:
df[df.min_age_limit < 45]

,date,min_age_limit,name,state_name,district_name,block_name,pincode,fee_type
